In [1]:
import os
os.chdir("../")
%pwd

'd:\\Machine_Learning\\Consignment-Pricing-Prediction'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [3]:
from ConsignmentPricingPrediction.utils.common import read_yaml, create_directories
from ConsignmentPricingPrediction.constants import *

In [4]:
class ConfigurationManager:
    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH,
            ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion 

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir,
        )
        return data_ingestion_config
        

In [5]:
import urllib.request as request
import zipfile
from ConsignmentPricingPrediction.logging import Logger

In [11]:
class DataIngestion:
    def __init__(self, data_ingestion_config: DataIngestionConfig):
        self.config = data_ingestion_config

    def download_file(self):
        if os.path.exists(self.config.local_data_file):
            Logger.info(f"{self.config.local_data_file} already exists")
        else:
            filename, header = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            Logger.info(f"File Downloaded:{filename} | \nHeaders: {header}")

    def extract_zip_file(self):

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
    
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

        Logger.info(f"{unzip_path} extracted")

    def initiate_data_ingestion(self):
        self.download_file()
        self.extract_zip_file()


In [12]:
# Data Ingestion Pipeline --------------------------------
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.initiate_data_ingestion()
except Exception as e:
    Logger.exception(e)

[2024-01-23 19:05:50,615: INFO: common: yaml file config\config.yaml loaded successfully]
[2024-01-23 19:05:50,618: INFO: common: yaml file params.yaml loaded successfully]
[2024-01-23 19:05:50,620: INFO: common: created directory at: artifacts]
[2024-01-23 19:05:50,622: INFO: common: created directory at: artifacts/data_ingestion]
[2024-01-23 19:05:50,623: INFO: 3752530512: artifacts/data_ingestion/data.zip already exists]
[2024-01-23 19:05:50,639: INFO: 3752530512: artifacts/data_ingestion extracted]
